# Match-Up

Given a SENTINEL-3 OLCI, i.e. [S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004](https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004) with specified attributes:

- `geometry`,
- [`properties.start_datetime`, `properties.end_datetime`]

search all active AERONET acquisitions, where:

- `properties.start_datetime` - 1h <= `Date(dd:mm:yyyy),Time(hh:mm:ss)` <= `properties.end_datetime` + 1h
- `Site_Latitude(Degrees),Site_Longitude(Degrees)` in `geometry`

## Execute the `search` operation

In [1]:
from datetime import (
    datetime,
    timedelta,
    timezone
)
from dateutil import parser
from pathlib import Path
from pystac import Item

import json
import sys

out_dir: Path = Path('.')

item: Item = Item.from_file('https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004')

def _parse_date_from_properties(date_string_id: str) -> datetime:
    return parser.parse(item.properties[date_string_id])

start_datetime = _parse_date_from_properties('start_datetime') - timedelta(hours=1)
end_datetime = _parse_date_from_properties('end_datetime') + timedelta(hours=1)

def _serialize_date(input_date: datetime) -> str:
    dt_utc = input_date.astimezone(timezone.utc)
    return dt_utc.isoformat().replace("+00:00", "Z")

cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": _serialize_date(start_datetime)},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": _serialize_date(end_datetime)},
            ],
        },
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                item.geometry,
            ],
        }
    ],
}

json.dump(cql2_filter, sys.stdout, indent=2)

{
  "op": "and",
  "args": [
    {
      "op": "eq",
      "args": [
        {
          "property": "data_type"
        },
        "AOD10"
      ]
    },
    {
      "op": "eq",
      "args": [
        {
          "property": "format"
        },
        "csv"
      ]
    },
    {
      "op": "eq",
      "args": [
        {
          "property": "data_format"
        },
        "daily-average"
      ]
    },
    {
      "op": "t_after",
      "args": [
        {
          "property": "time"
        },
        {
          "timestamp": "2025-10-31T08:47:06.893754Z"
        }
      ]
    },
    {
      "op": "t_before",
      "args": [
        {
          "property": "time"
        },
        {
          "timestamp": "2025-10-31T10:50:06.893754Z"
        }
      ]
    },
    {
      "op": "s_intersects",
      "args": [
        {
          "property": "geometry"
        },
        {
          "type": "Polygon",
          "coordinates": [
            [
              [
                -1.51

In [2]:
from pygeofilter_aeronet import aeronet_search

item: Item = aeronet_search(
    cql2_filter=cql2_filter,
    output_dir=out_dir,
    verbose=True
)

json.dump(item.to_dict(), sys.stdout, indent=2)

2025-11-13 16:18:13.905 | WARNING  | pygeofilter_aeronet.utils:wrapper:71 - GET https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?year=2025&month=10&day=31&AVG=20&AOD10=1&year2=2025&month2=10&day2=31&hour=8&hour2=10&lat1=39.545&lon1=-1.5187599999999999&lat2=52.45349999999999&lon2=18.484&if_no_html=1
2025-11-13 16:18:13.906 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > Host: aeronet.gsfc.nasa.gov
2025-11-13 16:18:13.907 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > Accept: */*
2025-11-13 16:18:13.907 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > Accept-Encoding: gzip, deflate
2025-11-13 16:18:13.907 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > Connection: keep-alive
2025-11-13 16:18:13.908 | WARNING  | pygeofilter_aeronet.utils:wrapper:75 - > User-Agent: python-httpx/0.28.1
2025-11-13 16:18:13.908 | WARNING  | pygeofilter_aeronet.utils:wrapper:77 - >
2025-11-13 16:18:48.544 | SUCCESS  | pygeofilter_aeronet.utils:wrapper:99 - < 200 OK
2025-11-13 16:

{
  "type": "Feature",
  "stac_version": "1.1.0",
  "stac_extensions": [],
  "id": "urn:uuid:bff1e343-32f1-4f5e-9139-709d47586845",
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -170.5644,
          -64.239996
        ],
        [
          174.768143,
          -64.239996
        ],
        [
          174.768143,
          68.661089
        ],
        [
          -170.5644,
          68.661089
        ],
        [
          -170.5644,
          -64.239996
        ]
      ]
    ]
  },
  "bbox": [
    -170.5644,
    -64.239996,
    174.768143,
    68.661089
  ],
  "properties": {
    "datetime": "2025-11-13T16:18:48.623460Z"
  },
  "links": [
    {
      "rel": "related",
      "href": "https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?AOD10=1&if_no_html=1&AVG=20&year=2025&month=10&day=31&hour=8&year2=2025&month2=10&day2=31&hour2=10&lon1=-1.5187599999999999&lat1=39.545&lon2=18.484&lat2=52.45349999999999",
      "type": "text/csv",
      "

## Visualize the results as Data Frame

In [3]:
from geopandas import read_parquet
from geopandas.geodataframe import GeoDataFrame

geoparquet_file: str = item.get_assets()['geoparquet'].href
geoparquet_data: GeoDataFrame = read_parquet(geoparquet_file)

geoparquet_data

,AERONET_Site,Date(dd:mm:yyyy),Time(hh:mm:ss),Day_of_Year,AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_865nm,AOD_779nm,AOD_675nm,...,N[500-870_Angstrom_Exponent],N[340-440_Angstrom_Exponent],N[440-675_Angstrom_Exponent[Polar]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m),geometry
0,Tucson,30:10:2025,12:00:00,303,0.022759,0.030613,0.032073,-999.0,-999.0,0.034822,...,99,99,0,lev10,1171,Tucson,32.233002,-110.953003,779.000,POINT (-110.953 32.233)
1,Tucson,31:10:2025,12:00:00,304,0.060602,0.065395,0.065430,-999.0,-999.0,0.067265,...,96,96,0,lev10,1171,Tucson,32.233002,-110.953003,779.000,POINT (-110.953 32.233)
2,GSFC,30:10:2025,12:00:00,303,0.007240,0.016739,0.022338,-999.0,-999.0,0.033286,...,32,32,0,lev10,1072,GSFC,38.992500,-76.839833,87.000,POINT (-76.83983 38.9925)
3,GSFC,31:10:2025,12:00:00,304,0.012323,0.020479,0.024331,-999.0,-999.0,0.031947,...,82,82,0,lev10,1072,GSFC,38.992500,-76.839833,87.000,POINT (-76.83983 38.9925)
4,Wallops,30:10:2025,12:00:00,303,0.033409,0.050313,0.056961,-999.0,-999.0,0.073485,...,1,1,0,lev10,1490,Wallops,37.932850,-75.471950,37.000,POINT (-75.47195 37.93285)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,Jeonju_RDA,30:10:2025,12:00:00,303,1.320580,1.250719,1.240559,-999.0,-999.0,1.241184,...,37,37,0,lev10,1535,Jeonju_RDA,35.828371,127.055012,42.174,POINT (127.05501 35.82837)
626,Invercargill_TROPOS,30:10:2025,12:00:00,303,0.043877,0.052294,0.056676,-999.0,-999.0,0.061928,...,20,18,0,lev10,1378,Invercargill_TROPOS,-46.417479,168.330753,5.000,POINT (168.33075 -46.41748)
627,Invercargill_TROPOS,31:10:2025,12:00:00,304,0.091755,0.093959,0.104787,-999.0,-999.0,0.102952,...,2,1,0,lev10,1378,Invercargill_TROPOS,-46.417479,168.330753,5.000,POINT (168.33075 -46.41748)
628,CIBA,30:10:2025,12:00:00,303,0.484599,0.444755,0.428063,-999.0,-999.0,0.410291,...,7,7,0,lev10,383,CIBA,41.814000,-4.932400,839.000,POINT (-4.9324 41.814)


## Visualize results on Map screen

In [4]:
from folium import (
    GeoJson,
    LayerControl,
    Map
)
from folium.plugins import (
    Fullscreen
)
from IPython.display import (
    display,
    HTML
)

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()
style = {"fillColor": "#00000000", "color": "#0000ff", "weight": 1}

footprints: GeoJson = GeoJson(
    geoparquet_data.dissolve(by='AERONET_Site').to_json(),
    name="Stac Item footprints",
    style_function=lambda x: style,
    control=True,
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map